# Training Pipeline
[run_training_dpo_pipeline.ipynb](https://github.com/shibing624/MedicalGPT/blob/main/run_training_dpo_pipeline.ipynb)    | [Open In Colab](https://colab.research.google.com/github/shibing624/MedicalGPT/blob/main/run_training_dpo_pipeline.ipynb)

# Stage 1: Continue Pretraining

第一阶段：PT(Continue PreTraining)增量预训练，在海量领域文本数据上二次预训练GPT模型，以适配领域数据分布

注意：
1. 此阶段是可选的，如果你没有海量领域文本，可以跳过此阶段，直接进行SFT阶段的有监督微调
2. 我实验发现：做领域知识注入，SFT比PT更高效，也可以跳过PT阶段

| Stage 1: Continue Pretraining   |  [pretraining.py](https://github.com/shibing624/MedicalGPT/blob/main/pretraining.py) | [run_pt.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_pt.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m`
2. 数据集：PT阶段使用的是中文天龙八部小说部分文本和英文书籍部分文本，位于`data/pretrain`文件夹

## 配置运行环境

本地执行可注释以下配置环境的命令，colab执行要打开注释，用于配置环境

colab建议使用T4 GPU训练，设置方式：`代码执行程序 -> 更改运行时类型 -> 运行时类型：Python3，硬件加速器：GPU，GPU类型：T4 -> 保存`

步骤：
1. 下载最新代码到本地
2. 安装依赖包

依赖包如下，保证最新版本：

```
loguru
transformers
sentencepiece
datasets
tensorboard
tqdm
peft
trl
```

In [1]:
!git clone --depth 1 https://github.com/shibing624/MedicalGPT.git
%cd MedicalGPT
%ls
!pip install -r requirements.txt

Cloning into 'MedicalGPT'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 69 (delta 11), reused 38 (delta 1), pack-reused 0
Receiving objects: 100% (69/69), 8.07 MiB | 12.10 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/MedicalGPT
build_domain_tokenizer.py          dpo_training.py             README.md
chatpdf.py                         fastapi_server_demo.py      requirements.txt
CITATION.cff                       gradio_demo.py              reward_modeling.py
_config.yml                        inference_multigpu_demo.py  run_dpo.sh
CONTRIBUTING.md                    inference.py                run_ppo.sh
convert_dataset.py                 LICENSE                     run_pt.sh
data/                              merge_peft_adapter.py       run_rm.sh
deepspeed_zero_stage2_config.json  merge_tokenizers.py         run_sft.sh
deepspeed_zero_stage3_config.json  ppo_training.p

## Stage1 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

**以下参数可以根据你的GPU实际情况修改，当前参数是根据Colab的T4单卡GPU（16GB显存）配置的**

In [2]:
%ls ./data/pretrain/

en_article_tail500.txt  fever.txt  tianlongbabu.txt


In [3]:
!python pretraining.py \
    --model_type bloom \
    --model_name_or_path bigscience/bloomz-560m \
    --train_file_dir ./data/pretrain \
    --validation_file_dir ./data/pretrain \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 3 \
    --do_train \
    --do_eval \
    --use_peft True \
    --seed 42 \
    --fp16 \
    --max_train_samples 20000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --block_size 128 \
    --group_by_length True \
    --output_dir outputs-pt-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2024-03-08 05:39:22.691589: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 05:39:22.691665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 05:39:22.693016: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 05:39:23.706874: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-08 05:39:24.452 | INFO     | __main__:main:378 - Model args: ModelArguments(model_type='bloom', model_name_or_path='bigscience/bloomz-560m', tokenizer_name_or_path=None, load_in_8bit=False, load_in_4bit=False, cache_dir=None, model_r

In [4]:
%ls -lh outputs-pt-v1

total 26M
-rw-r--r-- 1 root root  670 Mar  8 05:42 adapter_config.json
-rw-r--r-- 1 root root  13M Mar  8 05:42 adapter_model.safetensors
-rw-r--r-- 1 root root  435 Mar  8 05:42 all_results.json
drwxr-xr-x 2 root root 4.0K Mar  8 05:41 checkpoint-500/
-rw-r--r-- 1 root root  263 Mar  8 05:42 eval_results.json
-rw-r--r-- 1 root root 5.0K Mar  8 05:42 README.md
drwxr-xr-x 3 root root 4.0K Mar  8 05:39 runs/
-rw-r--r-- 1 root root  552 Mar  8 05:42 special_tokens_map.json
-rw-r--r-- 1 root root 1004 Mar  8 05:42 tokenizer_config.json
-rw-r--r-- 1 root root  14M Mar  8 05:42 tokenizer.json
-rw-r--r-- 1 root root  18K Mar  8 05:42 trainer_state.json
-rw-r--r-- 1 root root  192 Mar  8 05:42 train_results.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [5]:
!python merge_peft_adapter.py --model_type bloom \
    --base_model bigscience/bloomz-560m --lora_model outputs-pt-v1 --output_dir merged-pt/

Namespace(model_type='bloom', base_model='bigscience/bloomz-560m', tokenizer_path=None, lora_model='outputs-pt-v1', resize_emb=False, output_dir='merged-pt/')
Base model: bigscience/bloomz-560m
LoRA model: outputs-pt-v1
Loading LoRA for causal language model
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Merging with merge_and_unload...
Saving to Hugging Face format...
Done! model saved to merged-pt/


In [6]:
%ls -lh merged-pt/

total 1.1G
-rw-r--r-- 1 root root  807 Mar  8 05:42 config.json
-rw-r--r-- 1 root root  132 Mar  8 05:42 generation_config.json
-rw-r--r-- 1 root root 1.1G Mar  8 05:42 pytorch_model.bin
-rw-r--r-- 1 root root  552 Mar  8 05:42 special_tokens_map.json
-rw-r--r-- 1 root root  983 Mar  8 05:42 tokenizer_config.json
-rw-r--r-- 1 root root  14M Mar  8 05:42 tokenizer.json


In [7]:
%cat merged-pt/config.json

{
  "_name_or_path": "bigscience/bloomz-560m",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "masked_softmax_fusion": true,
  "model_type": "bloom",
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "offset_alibi": 100,
  "pad_token_id": 3,
  "pretraining_tp": 1,
  "seq_length": 2048,
  "skip_bias_add": true,
  "skip_bias_add_qkv": false,
  "slow_but_exact": false,
  "torch_dtype": "float16",
  "transformers_version": "4.38.2",
  "unk_token_id": 0,
  "use_cache": true,
  "vocab_size": 250880
}


Stage1 增量预训练完成。

# Stage 2: Supervised FineTuning

第二阶段：SFT(Supervised Fine-tuning)有监督微调，构造指令微调数据集，在预训练模型基础上做指令精调，以对齐指令意图，并注入领域知识

| Stage 2: Supervised Fine-tuning | [supervised_finetuning.py](https://github.com/shibing624/MedicalGPT/blob/main/supervised_finetuning.py) | [run_sft.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_sft.sh)  |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m` 或者 Stage1得到的预训练模型
2. 数据集：SFT阶段使用的是使用的是Belle的1千条抽样数据，位于`data/finetune`文件夹

## Stage2 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [8]:
%ls ./data/finetune

medical_sft_1K_format.jsonl  sharegpt_zh_1K_format.jsonl


In [9]:
!python supervised_finetuning.py \
    --model_type bloom \
    --model_name_or_path merged-pt \
    --train_file_dir ./data/finetune \
    --validation_file_dir ./data/finetune \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --do_train \
    --do_eval \
    --use_peft True \
    --fp16 \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --num_train_epochs 1 \
    --learning_rate 2e-5 \
    --warmup_ratio 0.05 \
    --weight_decay 0.05 \
    --logging_strategy steps \
    --logging_steps 10 \
    --eval_steps 50 \
    --evaluation_strategy steps \
    --save_steps 500 \
    --save_strategy steps \
    --save_total_limit 3 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 1 \
    --output_dir outputs-sft-v1 \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --device_map auto \
    --report_to tensorboard \
    --ddp_find_unused_parameters False \
    --gradient_checkpointing True

2024-03-08 05:43:25.893021: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 05:43:25.893079: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 05:43:25.894973: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 05:43:27.404553: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-08 05:43:28.221 | WARNING  | __main__:__post_init__:222 - You may set max_train_samples = -1 to run all samples in production.
2024-03-08 05:43:28.245 | INFO     | __main__:main:983 - Model args: ModelArguments(model_type='bloom', mo

In [10]:
%ls -lh outputs-sft-v1

total 26M
-rw-r--r-- 1 root root  657 Mar  8 05:47 adapter_config.json
-rw-r--r-- 1 root root  13M Mar  8 05:47 adapter_model.safetensors
-rw-r--r-- 1 root root  391 Mar  8 05:47 all_results.json
-rw-r--r-- 1 root root  222 Mar  8 05:47 eval_results.json
-rw-r--r-- 1 root root 5.0K Mar  8 05:47 README.md
drwxr-xr-x 3 root root 4.0K Mar  8 05:43 runs/
-rw-r--r-- 1 root root  552 Mar  8 05:47 special_tokens_map.json
-rw-r--r-- 1 root root 1004 Mar  8 05:47 tokenizer_config.json
-rw-r--r-- 1 root root  14M Mar  8 05:47 tokenizer.json
-rw-r--r-- 1 root root 5.7K Mar  8 05:47 trainer_state.json
-rw-r--r-- 1 root root  189 Mar  8 05:47 train_results.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [11]:
!python merge_peft_adapter.py --model_type bloom \
    --base_model merged-pt --lora_model outputs-sft-v1 --output_dir ./merged-sft

Namespace(model_type='bloom', base_model='merged-pt', tokenizer_path=None, lora_model='outputs-sft-v1', resize_emb=False, output_dir='./merged-sft')
Base model: merged-pt
LoRA model: outputs-sft-v1
Loading LoRA for causal language model
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Merging with merge_and_unload...
Saving to Hugging Face format...
Done! model saved to ./merged-sft


In [12]:
%ls -lh merged-sft/

total 1.1G
-rw-r--r-- 1 root root  794 Mar  8 05:47 config.json
-rw-r--r-- 1 root root  132 Mar  8 05:47 generation_config.json
-rw-r--r-- 1 root root 1.1G Mar  8 05:47 pytorch_model.bin
-rw-r--r-- 1 root root  552 Mar  8 05:47 special_tokens_map.json
-rw-r--r-- 1 root root  983 Mar  8 05:47 tokenizer_config.json
-rw-r--r-- 1 root root  14M Mar  8 05:47 tokenizer.json


In [13]:
%cat merged-sft/config.json

{
  "_name_or_path": "merged-pt",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "masked_softmax_fusion": true,
  "model_type": "bloom",
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "offset_alibi": 100,
  "pad_token_id": 3,
  "pretraining_tp": 1,
  "seq_length": 2048,
  "skip_bias_add": true,
  "skip_bias_add_qkv": false,
  "slow_but_exact": false,
  "torch_dtype": "float16",
  "transformers_version": "4.38.2",
  "unk_token_id": 0,
  "use_cache": true,
  "vocab_size": 250880
}


Stage2 SFT训练完成。

# Stage 3: DPO(Direct Preference Optimization)

第三阶段：DPO(Direct Preference Optimization)直接偏好优化，DPO通过直接优化语言模型来实现对其行为的精确控制，而无需使用复杂的强化学习，也可以有效学习到人类偏好，DPO相较于RLHF更容易实现且易于训练，效果更好

| Stage 3: Direct Preference Optimization        |  [dpo_training.py](https://github.com/shibing624/MedicalGPT/blob/main/dpo_training.py) | [run_dpo.sh](https://github.com/shibing624/MedicalGPT/blob/main/run_dpo.sh)    |

#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m` 或者 Stage2得到的SFT模型
2. 数据集：DPO阶段使用的是医疗reward数据，抽样了500条，位于`data/reward`文件夹

## Stage3 咱们开始吧

训练步骤如下：

1. 确认训练集
2. 执行训练脚本

训练脚本的执行逻辑如下：
1. 导入依赖包
2. 设置参数
3. 定义各函数并加载训练集
4. 加载模型和tokenizer
5. 开始训练并评估
6. 查看训练结果

In [14]:
%ls ./data/reward/

test.json


In [15]:
!python dpo_training.py \
    --model_type bloom \
    --model_name_or_path ./merged-sft \
    --train_file_dir ./data/reward \
    --validation_file_dir ./data/reward \
    --per_device_train_batch_size 3 \
    --per_device_eval_batch_size 1 \
    --do_train \
    --do_eval \
    --use_peft True \
    --max_train_samples 1000 \
    --max_eval_samples 10 \
    --max_steps 100 \
    --eval_steps 10 \
    --save_steps 50 \
    --max_source_length 128 \
    --max_target_length 128 \
    --output_dir outputs-dpo-v1 \
    --target_modules all \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --fp16 True \
    --device_map auto \
    --report_to tensorboard \
    --remove_unused_columns False \
    --gradient_checkpointing True \
    --cache_dir ./cache

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
2024-03-08 05:48:17.617605: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 05:48:17.617669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 05:48:17.619000: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 05:48:18.808848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could 

In [16]:
%ls -lh outputs-dpo-v1

total 26M
-rw-r--r-- 1 root root  660 Mar  8 05:51 adapter_config.json
-rw-r--r-- 1 root root  13M Mar  8 05:51 adapter_model.safetensors
-rw-r--r-- 1 root root  730 Mar  8 05:51 all_results.json
drwxr-xr-x 2 root root 4.0K Mar  8 05:51 checkpoint-100/
drwxr-xr-x 2 root root 4.0K Mar  8 05:49 checkpoint-50/
-rw-r--r-- 1 root root  558 Mar  8 05:51 eval_results.json
-rw-r--r-- 1 root root 5.0K Mar  8 05:51 README.md
drwxr-xr-x 3 root root 4.0K Mar  8 05:48 runs/
-rw-r--r-- 1 root root  552 Mar  8 05:51 special_tokens_map.json
-rw-r--r-- 1 root root 1003 Mar  8 05:51 tokenizer_config.json
-rw-r--r-- 1 root root  14M Mar  8 05:51 tokenizer.json
-rw-r--r-- 1 root root  55K Mar  8 05:51 trainer_state.json
-rw-r--r-- 1 root root 4.8K Mar  8 05:51 training_args.bin
-rw-r--r-- 1 root root  194 Mar  8 05:51 train_results.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`，合并到base model的方法见`merge_peft_adapter.py`
- 日志保存在`output_dir/runs`目录下，可以使用tensorboard查看，启动tensorboard方式如下：`tensorboard --logdir output_dir/runs --host 0.0.0.0 --port 8009`

lora模型权重合并到base model，合并后的模型保存在`--output_dir`目录下，合并方法如下：

In [17]:
!python merge_peft_adapter.py --model_type bloom \
    --base_model merged-sft --lora_model outputs-dpo-v1 --output_dir merged-dpo/

Namespace(model_type='bloom', base_model='merged-sft', tokenizer_path=None, lora_model='outputs-dpo-v1', resize_emb=False, output_dir='merged-dpo/')
Base model: merged-sft
LoRA model: outputs-dpo-v1
Loading LoRA for causal language model
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Merging with merge_and_unload...
Saving to Hugging Face format...
Done! model saved to merged-dpo/


In [18]:
%ls -lh merged-dpo/

total 1.1G
-rw-r--r-- 1 root root  795 Mar  8 05:51 config.json
-rw-r--r-- 1 root root  132 Mar  8 05:51 generation_config.json
-rw-r--r-- 1 root root 1.1G Mar  8 05:51 pytorch_model.bin
-rw-r--r-- 1 root root  552 Mar  8 05:51 special_tokens_map.json
-rw-r--r-- 1 root root  983 Mar  8 05:51 tokenizer_config.json
-rw-r--r-- 1 root root  14M Mar  8 05:51 tokenizer.json


In [19]:
%cat merged-dpo/config.json

{
  "_name_or_path": "merged-sft",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "masked_softmax_fusion": true,
  "model_type": "bloom",
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "offset_alibi": 100,
  "pad_token_id": 3,
  "pretraining_tp": 1,
  "seq_length": 2048,
  "skip_bias_add": true,
  "skip_bias_add_qkv": false,
  "slow_but_exact": false,
  "torch_dtype": "float16",
  "transformers_version": "4.38.2",
  "unk_token_id": 0,
  "use_cache": true,
  "vocab_size": 250880
}


Stage3 偏好建模第一次训练完成。

**至此一个完整的训练流程演示完成。**

# Test

In [ ]:
!python inference.py --model_type bloom --base_model merged-dpo --interactive

2024-03-08 05:52:29.991003: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 05:52:29.991059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 05:52:29.992387: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 05:52:31.607767: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Namespace(model_type='bloom', base_model='merged-dpo', lora_model='', tokenizer_path=None, template_name='vicuna', repetition_penalty=1.0, max_new_tokens=512, data_file=None, interactive=True, single_tune=False, do_sample=False, output_file=

Input:介绍下南京
Response:  南京市位于江苏省西南部，是全国首批历史文化名城、国家中心城市和自由贸易试验区。

完。
